Define initial condition (load from data, 1 or 2 sims), define grid. 
Boundary conditions? Periodic in theta, what about inflow and outflow?

In [1]:
from clawpack import riemann

import matplotlib.pyplot as plt

from clawpack import pyclaw
from clawpack.pyclaw.examples.advection_2d_annulus.mapc2p import mapc2p

import numpy as np
from clawpack.visclaw import colormaps

import re

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
r_lower = 3.903302085636277
r_upper = 23.465031329617336
m_r = 126

theta_lower = 0.0
theta_upper = np.pi*2.0
m_theta = 512

In [3]:
def mapc2p_annulus(xc, yc):
    """
    Specifies the mapping to curvilinear coordinates.

    Inputs: c_centers = Computational cell centers
                 [array ([Xc1, Xc2, ...]), array([Yc1, Yc2, ...])]

    Output: p_centers = Physical cell centers
                 [array ([Xp1, Xp2, ...]), array([Yp1, Yp2, ...])]
    """  
    p_centers = []

    # Polar coordinates (first coordinate = radius,  second coordinate = theta)
    p_centers.append(xc[:]*np.cos(yc[:]))
    p_centers.append(xc[:]*np.sin(yc[:]))
    
    return p_centers

In [4]:
raw_data = np.load("/home/ajivani/WLROM_new/WhiteLight/validation_data/CR2161_validation_PolarTensor.npy")
raw_data.shape

sample_rd = raw_data[:126, :, 25, 20]

In [12]:
sample_rd.shape

(126, 512)

In [ ]:
def setup(use_petsc=False,outdir='./_output',solver_type='classic'):
    from clawpack import riemann

    if use_petsc:
        import clawpack.petclaw as pyclaw
    else:
        from clawpack import pyclaw

    if solver_type == 'classic':
        solver = pyclaw.ClawSolver2D(riemann.vc_advection_2D)
        solver.dimensional_split = False
        solver.transverse_waves = 2
        solver.order = 2
    elif solver_type == 'sharpclaw':
        solver = pyclaw.SharpClawSolver2D(riemann.vc_advection_2D)

    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    solver.bc_lower[1] = pyclaw.BC.periodic
    solver.bc_upper[1] = pyclaw.BC.periodic

    solver.aux_bc_lower[0] = pyclaw.BC.custom
    solver.aux_bc_upper[0] = pyclaw.BC.custom
    solver.user_aux_bc_lower = ghost_velocities_lower
    solver.user_aux_bc_upper = ghost_velocities_upper
    solver.aux_bc_lower[1] = pyclaw.BC.periodic
    solver.aux_bc_upper[1] = pyclaw.BC.periodic

    solver.dt_initial = 0.1
    solver.cfl_max = 0.5
    solver.cfl_desired = 0.4

    solver.limiters = pyclaw.limiters.tvd.vanleer

#     r_lower = 0.2
#     r_upper = 1.0
#     m_r = 40

#     theta_lower = 0.0
#     theta_upper = np.pi*2.0
#     m_theta = 120

    r_lower = 3.903302085636277
    r_upper = 23.465031329617336
    m_r = 126

    theta_lower = 0.0
    theta_upper = np.pi*2.0
    m_theta = 512

    r     = pyclaw.Dimension(r_lower,r_upper,m_r,name='r')
    theta = pyclaw.Dimension(theta_lower,theta_upper,m_theta,name='theta')
    domain = pyclaw.Domain([r,theta])
    domain.grid.mapc2p = mapc2p_annulus
    domain.grid.num_ghost = solver.num_ghost

    num_eqn = 1
    state = pyclaw.State(domain,num_eqn)

    qinit(state)

    dx, dy = state.grid.delta
    p_corners = state.grid.p_nodes
    state.aux = edge_velocities_and_area(p_corners[0],p_corners[1],dx,dy)
    state.index_capa = 2 # aux[2,:,:] holds the capacity function

    claw = pyclaw.Controller()
    claw.tfinal = 1.0
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.outdir = outdir
    claw.setplot = setplot
    claw.keep_copy = True

    return claw

In [9]:
def qinit(state):
    """
    Initialize with two Gaussian pulses.
    """
#     # First gaussian pulse
#     A1     = 1.    # Amplitude
#     beta1  = 40.   # Decay factor
#     r1     = -0.5  # r-coordinate of the center
#     theta1 = 0.    # theta-coordinate of the center

#     # Second gaussian pulse
#     A2     = -1.   # Amplitude
#     beta2  = 40.   # Decay factor
#     r2     = 0.5   # r-coordinate of the centers
#     theta2 = 0.    # theta-coordinate of the centers

    R, Theta = state.grid.p_centers
    state.q[0, :, :] = sample_rd
#     state.q[0,:,:] = A1*np.exp(-beta1*(np.square(R-r1) + np.square(Theta-theta1)))\
#                    + A2*np.exp(-beta2*(np.square(R-r2) + np.square(Theta-theta2)))

In [10]:
from clawpack import riemann

use_petsc = False

if use_petsc:
    import clawpack.petclaw as pyclaw
else:
    from clawpack import pyclaw

solver_type = 'classic'
    
if solver_type == 'classic':
    solver = pyclaw.ClawSolver2D(riemann.vc_advection_2D)
    solver.dimensional_split = False
    solver.transverse_waves = 2
    solver.order = 2
elif solver_type == 'sharpclaw':
    solver = pyclaw.SharpClawSolver2D(riemann.vc_advection_2D)


r_lower = 3.903302085636277
r_upper = 23.465031329617336
m_r = 126

theta_lower = 0.0
theta_upper = np.pi*2.0
m_theta = 512

r     = pyclaw.Dimension(r_lower,r_upper,m_r,name='r')
theta = pyclaw.Dimension(theta_lower,theta_upper,m_theta,name='theta')
domain = pyclaw.Domain([r,theta])
domain.grid.mapc2p = mapc2p_annulus
domain.grid.num_ghost = solver.num_ghost

num_eqn = 1
state = pyclaw.State(domain,num_eqn)

qinit(state)

In [11]:
# state.q[0, :, :].shape

(126, 512)